# Probabilistic State Estimation

> This tutorial is implementation for [MIT state estimation class](https://ocw.mit.edu/courses/6-01sc-introduction-to-electrical-engineering-and-computer-science-i-spring-2011/pages/unit-4-probability-and-planning/state-estimation/)

In [ ]:
#| default_exp probabilistic_state_estimation

In [ ]:
#| export
class DDist:
    def __init__(self, dictionary):
        self.d = dictionary

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()